In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(2024)

## Model

In [2]:
vocab_size = 500
model_dimension = 128

In [50]:
class NormalizationLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):

        return F.normalize(x, dim=1)

class BaseNetwork(nn.Module):
    def __init__(self, vocab_size, model_dimension):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, model_dimension)
        self.lstm = nn.LSTM(model_dimension, model_dimension, batch_first=True)
        self.pooling = nn.AvgPool1d(kernel_size=2)
        self.l2norm = NormalizationLayer()

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = torch.transpose(x, 1, 2)
        x = self.pooling(x)
        x = self.l2norm(x)

        return x

class SiameseNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.LSTM = BaseNetwork(vocab_size, model_dimension)

    def forward(self, input1, input2):
        output1 = self.LSTM(input1)
        output2 = self.LSTM(input2)
        conc = torch.cat((output1, output2), dim=1)

        return conc

In [51]:
siamese_nn = SiameseNetwork()
LSTM = BaseNetwork(vocab_size, model_dimension)

inputs1 = torch.randint(0, vocab_size, (32, 100))
inputs2 = torch.randint(0, vocab_size, (32, 100))

outputs = siamese_nn(inputs1, inputs2)

In [52]:
def show_layers(model, layer_prefix=''):
    def recurse(module, prefix=''):
        for name, child in module.named_children():
            class_name = child.__class__.__name__
            print(f'{prefix}{name}: {class_name}')
            if list(child.children()):  # if the child has children, recurse
                recurse(child, prefix + '  ')

    print(f"Model: {model.__class__.__name__}")
    print("=" * 50)
    recurse(model, layer_prefix)
    print("\nTotal parameters:", sum(p.numel() for p in model.parameters()))

# Usage:
print('Siamese model:')
show_layers(siamese_nn, 'SiameseNetwork.')

print('\nDetail of Base Network:')
show_layers(LSTM, 'BaseNetwork.')

Siamese model:
Model: SiameseNetwork
SiameseNetwork.LSTM: BaseNetwork
SiameseNetwork.  embedding: Embedding
SiameseNetwork.  lstm: LSTM
SiameseNetwork.  pooling: AvgPool1d
SiameseNetwork.  l2norm: NormalizationLayer

Total parameters: 196096

Detail of Base Network:
Model: BaseNetwork
BaseNetwork.embedding: Embedding
BaseNetwork.lstm: LSTM
BaseNetwork.pooling: AvgPool1d
BaseNetwork.l2norm: NormalizationLayer

Total parameters: 196096
